In [88]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
# Parse a url and returns a soup of url
def get_requests_soup(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [ ]:
# Getting all league urls from understat

### Start
url = 'https://understat.com/'
soup = get_requests_soup(url)
links = soup.find_all('a', attrs={'class':'link'}, href=True)
league_urls = [url+tag['href'] for tag in links]

### Loop

for url in league_urls:
    soup = get_requests_soup(url)
    print(soup.text)
    break  # Break after 1 loop
print("Finished")



In [ ]:
## Unnecessary after dictionaries obtained from json
def get_team_names(soup):
    ## Get team names
    meta = soup.find_all('meta', attrs={'name':'Keywords'})
    keywords = [a['content'] for a in meta][0].split(',')
#     print(keywords)
    home_team, away_team, league, season, season_start_date = keywords[0], keywords[1], keywords[2], keywords[3], keywords[4]
    
    

In [58]:
## Unnecessary after dictionaries obtained from json
def get_xG(soup):    
    p_bars = soup.find_all('div', attrs={'class':'progress-bar'} )

    values = []
                                         
    for a in p_bars:
        if 'xG' in a.text:
            p_value = a.find_all('div', attrs={'class':'progress-value'})
            values = [val.get_text() for val in p_value]
            
    return values

In [117]:
# Get data from script tags found
# Input soup from specific match url
def get_json_tags(soup):
    script = soup.find_all("script", text=True)
    
    # Convert html json code into readable strings
    script_tags = [s.encode('utf8').decode('unicode_escape') for s in script if 'JSON' in s.encode('utf8').decode('unicode_escape')]
    
    # Remove unecessary characters and parse a clean string of the json 
    jsons = []
    for script_tag in script_tags:
        all_json = re.findall(r"JSON.parse\('([^']*)", script_tag)
        for j in all_json:
            jsons.append(j)
    
    # Declare some dictionaries
    shots_data = {}      # 0 index
    match_info = {}      # 1
    rosters_data = {}    # 2
#     promotion_data = {}  # 3 - No idea what this is
    
#     print(len(jsons))
    if jsons:
        shots_data = json.loads(jsons[0])
        match_info = json.loads(jsons[1])
        rosters_data = json.loads(jsons[2])
#         promotion_data = json.loads(jsons[3])
        
    return shots_data, match_info, rosters_data
        
    

In [119]:
def main():
    print("Main - Start")
    
    url = 'https://understat.com/match/14598'
    soup = get_requests_soup(url)
    
    shots_data, match_info, rosters_data = get_json_tags(soup)
    
    print(f'Home xG: {match_info["h_xg"]}')
    print(f'Away xG: {match_info["a_xg"]}')
    
    
    print("Main - Finished.")
    
main()

Main - Start
4
Home xG: 0.500436
Away xG: 1.37204
Main - Finished.
